In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
from core.consts import EIA_TEST_SET_HOURS, EIA_EARLIEST_HOUR_UTC
from core.data import get_local_dvc_git_repo, get_DvcRepo
from flows.etl_flow import etl, concurrent_fetch_EIA_data
from prefect import flow
from pathlib import Path
import os
import pandas as pd

In [58]:
local_dvc_repo_path = Path(os.getenv('DVC_LOCAL_REPO_PATH'))
git_repo = get_local_dvc_git_repo()
dvc_repo = get_DvcRepo()

In [66]:
chunk_idx_path = local_dvc_repo_path / 'v1/chunk_idx.parquet'
chunk_data_path = local_dvc_repo_path / 'v1/data'
idx_df = pd.read_parquet(chunk_idx_path)

In [67]:
idx_df

,year,quarter,start_ts,end_ts,data_start_ts,data_end_ts,name,complete
0,2015,3,2015-07-01 00:00:00+00:00,2015-09-30 23:00:00+00:00,2015-07-01 05:00:00+00:00,2015-09-30 23:00:00+00:00,2015_Q3_from_2015-07-01-00_to_2015-09-30-23,False
1,2015,4,2015-10-01 00:00:00+00:00,2015-12-31 23:00:00+00:00,2015-10-01 00:00:00+00:00,2015-12-31 23:00:00+00:00,2015_Q4_from_2015-10-01-00_to_2015-12-31-23,True
2,2016,1,2016-01-01 00:00:00+00:00,2016-03-31 23:00:00+00:00,2016-01-01 00:00:00+00:00,2016-03-31 23:00:00+00:00,2016_Q1_from_2016-01-01-00_to_2016-03-31-23,True
3,2016,2,2016-04-01 00:00:00+00:00,2016-06-30 23:00:00+00:00,2016-04-01 00:00:00+00:00,2016-06-30 23:00:00+00:00,2016_Q2_from_2016-04-01-00_to_2016-06-30-23,True
4,2016,3,2016-07-01 00:00:00+00:00,2016-09-30 23:00:00+00:00,2016-07-01 00:00:00+00:00,2016-09-30 23:00:00+00:00,2016_Q3_from_2016-07-01-00_to_2016-09-30-23,True
5,2016,4,2016-10-01 00:00:00+00:00,2016-12-31 23:00:00+00:00,2016-10-01 00:00:00+00:00,2016-12-31 23:00:00+00:00,2016_Q4_from_2016-10-01-00_to_2016-12-31-23,True
6,2017,1,2017-01-01 00:00:00+00:00,2017-03-31 23:00:00+00:00,2017-01-01 00:00:00+00:00,2017-03-31 23:00:00+00:00,2017_Q1_from_2017-01-01-00_to_2017-03-31-23,True
7,2017,2,2017-04-01 00:00:00+00:00,2017-06-30 23:00:00+00:00,2017-04-01 00:00:00+00:00,2017-06-30 23:00:00+00:00,2017_Q2_from_2017-04-01-00_to_2017-06-30-23,True
8,2017,3,2017-07-01 00:00:00+00:00,2017-09-30 23:00:00+00:00,2017-07-01 00:00:00+00:00,2017-09-30 23:00:00+00:00,2017_Q3_from_2017-07-01-00_to_2017-09-30-23,True
9,2017,4,2017-10-01 00:00:00+00:00,2017-12-31 23:00:00+00:00,2017-10-01 00:00:00+00:00,2017-12-31 23:00:00+00:00,2017_Q4_from_2017-10-01-00_to_2017-12-31-23,True


In [74]:
@flow
def fetch_data(start_ts, end_ts):
    return concurrent_fetch_EIA_data(start_ts, end_ts)

In [75]:
# Given a requested range
start_ts = pd.Timestamp('2024-01-15 10:00:00+00:00')
end_ts = pd.Timestamp('2024-10-30 00:00:00+00:00')
# Determine which parts of the range can be served by DVC chunks, and which must be fetched
# Assume the DVC warehouse chunks are contiguous, and we only append to the recent chunks.
dvc_data_start_ts = idx_df.iloc[0].data_start_ts
dvc_data_end_ts = idx_df.iloc[-1].data_end_ts

if start_ts < dvc_data_start_ts:
    # EIA doesn't serve data before what's already archived in DVC
    # In the future we could extract it from their csv archives and add it to DVC
    raise NotImplementedError
if end_ts > dvc_data_end_ts:
    fetch_start_ts = dvc_data_end_ts
    fetch_end_ts = end_ts
    print(f'Fetch recent data. from:{fetch_start_ts}. to:{fetch_end_ts}')
    df = fetch_data(fetch_start_ts, fetch_end_ts)
    print(df)

Fetch recent data. from:2024-10-28 00:00:00+00:00. to:2024-10-30 00:00:00+00:00


20:53:32.598 | INFO    | prefect.engine - Created flow run 'quick-cuttlefish' for flow 'fetch-data'

20:53:32.605 | INFO    | prefect.engine - View at https://app.prefect.cloud/account/914106c7-cc3b-4c5c-99f4-915f39352746/workspace/d3b27b3d-41ca-47a0-9b5b-2eebd92a4b25/runs/flow-run/04f92616-0035-47a8-8efc-31794256201f

20:53:32.970 | INFO    | Task run 'concurrent_fetch_EIA_data-145' - Created task run 'concurrent_fetch_EIA_data-145' for task 'concurrent_fetch_EIA_data'

Fetching API page. offset:0. length:5000
Total records to fetch: 49
('Fetching 48 hours of data: 49 records.\n', 'Start: 2024-10-28 00:00:00+00:00. End: 2024-10-30 00:00:00+00:00')
Will make 0 5000-length requests and one 49-length request.


...

Fetching API page. offset:0. length:49


20:53:36.297 | INFO    | Task run 'get_eia_data_as_df-741' - Finished in state Completed()

20:53:36.320 | INFO    | Task run 'concurrent_fetch_EIA_data-145' - Finished in state Completed()

20:53:36.507 | INFO    | Flow run 'quick-cuttlefish' - Finished in state Completed()

                          respondent           respondent-name type type-name  \
utc_ts                                                                          
2024-10-28 00:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 01:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 02:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 03:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 04:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 05:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 06:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 07:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 08:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 09:00:00+00:00        PJM  PJM Interconnection, LLC    D    Demand   
2024-10-28 10:00:00+00:00   